<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Seed_VC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First install Everything. The Restart Session require after install finished.

In [ ]:
!git clone https://huggingface.co/spaces/Plachta/Seed-VC seed-vc
%cd seed-vc
!sed -i "s/).launch()/).launch(share=True)/" app.py
!pip install -r requirements.txt
!pip install spaces protobuf==3.20.3 tensorflow==2.12.0

After everything installed, restart the session by pressing pop-up screen. After that, launch the app.py and enjoy. :)

In [ ]:
!python seed-vc/app.py